In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
from glob import glob
import statistics
import os
import pandas as pd
from scipy import stats

### CCH5000

Incremental Accuracy (Table 1 & Figure 4)

In [ ]:
folder = f'results/pretrain'
exps = {
    1:
       {
        'iCaRL': f'{folder}/icarl/20220728_icarl_colorectal_4steps1',
        'UCIR': f'{folder}/ucir/20220728_ucir_colorectal_4steps1',
        'PODNet': f'{folder}/podnet/20220728_podnet_colorectal_4steps1',
        'DER': f'{folder}/der/20220728_derE0_colorectal_4steps1',
        'Ours': f'{folder}/ours/20220731_derE3_colorectal_4steps1',
       },
    2:
       {
        'iCaRL': f'{folder}/icarl/20220728_icarl_colorectal_4steps2',
        'UCIR': f'{folder}/ucir/20220728_ucir_colorectal_4steps2',
        'PODNet': f'{folder}/podnet/20220728_podnet_colorectal_4steps2',
        'DER': f'{folder}/der/20220728_derE0_colorectal_4steps2',
        'Ours': f'{folder}/ours/20220731_derE3_colorectal_4steps2',
       }       
      }

incs=[1,2]
row = min(len(incs),3)
plt.figure(figsize=(8*len(incs),8))
for j, inc in enumerate(incs):
    print(f'Incremental class per step: {inc}')
    exp = exps[inc]
    plt.subplot(int(np.ceil(len(incs)//row)),row,j+1)

    avg_top1_total = {}
    cul_top1_total = {}
    count = {}
    for k in [0,1,2]:
        for e1, e2 in exp.items():
            file = f'{e2}/run_{k}_1993_.json'
            if not os.path.exists(file):
                continue
            result = json.load(open(file))

            top1_total = list(map(lambda x: 100 * x['accuracy']['total'], result['results']))
        
            if e1 in avg_top1_total:
                avg_top1_total[e1] += np.array(top1_total)
                cul_top1_total[e1].append(np.mean(top1_total))
                count[e1] += 1
            else:
                avg_top1_total[e1] = np.array(top1_total)
                cul_top1_total[e1] = [np.mean(top1_total)]
                count[e1] = 1
        
    for e1, e2 in avg_top1_total.items():
        top1_total = e2/count[e1]
        mean = statistics.mean(cul_top1_total[e1]) if len(cul_top1_total[e1])>1 else cul_top1_total[e1][0]
        std = statistics.stdev(cul_top1_total[e1]) if len(cul_top1_total[e1])>1 else 0
        if e1 != 'Ours':
            print(f"\t {e1}: {stats.ttest_rel(cul_top1_total['Ours'], cul_top1_total[e1])}")
        xs = list(range(4,9,inc))
        plt.plot(xs, top1_total, marker='o', label=f'{e1}: {mean:.1f}' + r' $\pm$ ' + f'{std:.1f}', linewidth=2, markersize=10)

    # plt.plot(xs, test)

    plt.xlabel('Number of Classes', fontsize=25)
    plt.xticks(fontsize=22)
    plt.ylabel('Accuracy', fontsize=25)
    plt.yticks(fontsize=22)
#     plt.ylim(71,89)
#     plt.title(f'{inc} new classes per step', fontsize=25)
    plt.grid(True)
    plt.legend(prop={'size':25})
#     plt.tight_layout()

plt.show()


Forgetting (Table 1 & Figure 4)

In [ ]:
folder = f'results/pretrain'
exps = {
    1:
       {
        'iCaRL': f'{folder}/icarl/20220728_icarl_colorectal_4steps1',
        'UCIR': f'{folder}/ucir/20220728_ucir_colorectal_4steps1',
        'PODNet': f'{folder}/podnet/20220728_podnet_colorectal_4steps1',
        'DER': f'{folder}/der/20220728_derE0_colorectal_4steps1',
        'Ours': f'{folder}/ours/20220731_derE3_colorectal_4steps1',
       },
    2:
       {
        'iCaRL': f'{folder}/icarl/20220728_icarl_colorectal_4steps2',
        'UCIR': f'{folder}/ucir/20220728_ucir_colorectal_4steps2',
        'PODNet': f'{folder}/podnet/20220728_podnet_colorectal_4steps2',
        'DER': f'{folder}/der/20220728_derE0_colorectal_4steps2',
        'Ours': f'{folder}/ours/20220731_derE3_colorectal_4steps2',
       }       
      }

incs=[1,2]
row = min(len(incs),3)
plt.figure(figsize=(9*len(incs),9))
for j, inc in enumerate(incs):
    print(f'Incremental class per step: {inc}')
    exp = exps[inc]
    step = int(4//inc)

    plt.subplot(int(np.ceil(len(incs)//row)),row,j+1)

    all_forget = {}
    avg_forget = {}
    count = {}
    for j in [0,1,2]:
        for e1, e2 in exp.items():
            file = f'{e2}/run_{j}_1993_.json'
            if not os.path.exists(file):
                continue
            result = json.load(open(file))
            
            all_forget_total = {}
            for k in range(step,step*2+1):
                if k == step:
                    start = 0
                else:
                    start = (k-1)*inc
                end = k*inc - 1

                top1_first = np.zeros(len(result['results']) - (k-step))
                for p in range(start, end+1):
                    new_key = f'{str(p).zfill(2)}-{str(p).zfill(2)}'
                    top1_first += np.array(list(map(lambda x: x['accuracy_per_class'][new_key], result['results'][(k-step):])))
                top1_first /= (end-start+1) / 100

                forgetting = np.concatenate([[0], np.maximum.accumulate(top1_first)[:-1] - top1_first[1:]])

                if e1 in all_forget_total:
                    all_forget_total[e1][k-step:] += np.array(forgetting)
                else:
                    all_forget_total[e1] = np.array(forgetting)
                    
            forgetting = all_forget_total[e1]        
            forgetting[1:] /= np.arange(len(forgetting)-1) + 1
            if e1 in all_forget:
                all_forget[e1] += forgetting
                avg_forget[e1].append(np.mean(forgetting))
                count[e1] += 1
            else:
                all_forget[e1] = forgetting
                avg_forget[e1] = [np.mean(forgetting)]
                count[e1] = 1
            
    for e1, e2 in all_forget.items():
        forgetting = all_forget[e1]/count[e1]
        mean = statistics.mean(avg_forget[e1]) if len(avg_forget[e1])>1 else avg_forget[e1][0]
        std = statistics.stdev(avg_forget[e1]) if len(avg_forget[e1])>1 else 0
        if e1 != 'Ours':
            print(f"\t {e1}: {stats.ttest_rel(avg_forget['Ours'], avg_forget[e1])}")
        xs = list(range(4,9,inc))
        plt.plot(xs, forgetting, marker='o', label=f'{e1}: {mean:.1f}' + r' $\pm$ ' + f'{std:.1f}', linewidth=2, markersize=10)
        
    plt.xlabel('Number of Classes', fontsize=25)
    plt.xticks(fontsize=22)
    plt.ylabel('Forgetting', fontsize=25)
    plt.yticks(fontsize=22)
    plt.grid(True)
#     plt.ylim(-1,15)
#     plt.title(f'{inc} new classes per step', fontsize=25)
    plt.legend(prop={'size':25})
#     plt.tight_layout()

plt.show()


Ablation study (Table 4)

In [ ]:
folder = f'results/pretrain'
exps = {
        'Ours': f'{folder}/ours/20220731_derE3_colorectal_4steps1',
        'Without alternating loss': f'{folder}/ablation/20220731_derE3_colorectal_4steps1_noAlt',
        'Without auxiliary loss': f'{folder}/ablation/20220731_derE3_colorectal_4steps1_noAux',
        'Without distillation loss': f'{folder}/ablation/20220731_derE3_colorectal_4steps1_noDis',
        'Without margin loss': f'{folder}/ablation/20220731_derE3_colorectal_4steps1_noMar',
       }

inc = 1
avg_top1_total = {}
cul_top1_total = {}
avg_top1_last = {}
avg_top1_first = {}
count = {}
for k in [0,1,2]:
    for e1, e2 in exps.items():
        file = f'{e2}/run_{k}_1993_.json'
        if not os.path.exists(file):
            continue
        result = json.load(open(file))

        top1_total = list(map(lambda x: 100 * x['accuracy']['total'], result['results']))
        top1_last = [result['results'][i]['accuracy'][f'0{4+(i-1)}-0{4+i-1}'] * 100 for i in range(1,5)]

        top1_first = np.zeros(5)
        for p in range(0,4,1):
            new_key = f'{str(p).zfill(2)}-{str(p).zfill(2)}'
            top1_first += np.array(list(map(lambda x: x['accuracy'][new_key], result['results'])))
        top1_first = top1_first / 4 * 100
        
        if e1 in avg_top1_total:
            avg_top1_total[e1] += np.array(top1_total)
            cul_top1_total[e1].append(np.mean(top1_total))
            avg_top1_last[e1].append(np.mean(top1_last))
            avg_top1_first[e1].append(np.mean(top1_first))
            count[e1] += 1
        else:
            avg_top1_total[e1] = np.array(top1_total)
            cul_top1_total[e1] = [np.mean(top1_total)]
            avg_top1_last[e1] = [np.mean(top1_last)]
            avg_top1_first[e1] = [np.mean(top1_first)]  
            count[e1] = 1

df = pd.DataFrame(columns={'Case','Setting','Incremental','New task','First task','Incremental std','New task std','First task std'})            
for i, (e1, e2) in enumerate(avg_top1_total.items()):
    top1_total = e2/count[e1]
    mean = statistics.mean(cul_top1_total[e1]) if len(cul_top1_total[e1])>1 else cul_top1_total[e1][0]
    std = statistics.stdev(cul_top1_total[e1]) if len(cul_top1_total[e1])>1 else 0
    mean_last = statistics.mean(avg_top1_last[e1]) if len(avg_top1_last[e1])>1 else avg_top1_last[e1][0]
    std_last = statistics.stdev(avg_top1_last[e1]) if len(avg_top1_last[e1])>1 else 0
    mean_first = statistics.mean(avg_top1_first[e1]) if len(avg_top1_first[e1])>1 else avg_top1_first[e1][0]
    std_first = statistics.stdev(avg_top1_first[e1]) if len(avg_top1_first[e1])>1 else 0
    xs = list(range(100,201,inc))
    title = e1.replace("\n  ",  "")
    print(f'{title}: \n  Average incremental accuracy: {mean:.1f}' + u' \u00B1 ' + f'{std:.1f}')
    print(f'  Average new task accuracy: {mean_last:.1f}' + u' \u00B1 ' + f'{std_last:.1f}')
    print(f'  Average first task accuracy: {mean_first:.1f}' + u' \u00B1 ' + f'{std_first:.1f}\n')
    df = df.append({'Case':f'Case {i}' if e1!='Ours' else e1,
                    'Setting':e1,
                    'Incremental':mean,
                    'Incremental std':std,
                    'New task':mean_last,
                    'New task std':std_last,
                    'First task':mean_first, 
                    'First task std':std_first}, 
                   ignore_index=True)

import matplotlib.patches as mpatches
ax = df.plot(x="Case", 
        y=['New task','Incremental','First task'],
        yerr=df[['New task std','Incremental std','First task std']].T.values,
        kind="bar",
        ylim=(65,105),
        rot=15,
        figsize=(14,6), 
        capsize=5,
        linewidth=0.5, 
        width=0.65,
        fontsize=18)    
for bar in ax.containers[3]:
    bar.set_hatch('..')
    
ax.set_xlabel('Ablation Setting',fontdict={'fontsize':20})
ax.set_ylabel('Average Accuracy',fontdict={'fontsize':20})
ax.legend(handles=[
    mpatches.Patch(color='#1f77b4', label='Average new task accuracy'),
    mpatches.Patch(facecolor='#ff7f0e', hatch='..', label='Average accuracy'),
    mpatches.Patch(color='#2ca02c', label='Average first task accuracy'),
], handleheight=1,fontsize=16, loc='upper center')    
plt.axhline(y=statistics.mean(cul_top1_total['Ours']), color='#ff7f0e', linestyle='--', linewidth=1.2)
txtstr = ''
for i, text in enumerate(exps.keys()):
    if i == 0:
        continue
    else:
        txtstr += '$\\bf{Case ' + str(i) + '}$' + f': \n  {text}\n'
plt.text(4.7, 80, txtstr, fontsize=20)
plt.show()

### EyePACS

Incremental Accuracy (Table 2)

In [ ]:
folder = f'results/pretrain'
exps = {
    1:
       {
        'iCaRL': f'{folder}/icarl/20220403_icarl_diabetic_3steps1',
        'UCIR': f'{folder}/ucir/20220403_ucir_diabetic_3steps1',
        'PODNet': f'{folder}/podnet/20220403_podnet_cnn_diabetic_3steps1',
        'DER': f'{folder}/der/20220404_derE0_diabetic_3steps1',
        'Ours': f'{folder}/ours/20220403_derE3_diabetic_3steps1',
       },
      }

incs=[1]
plt.figure(figsize=(6,6))

inc = 1
exp = exps[inc]

avg_top1_total = {}
cul_top1_total = {}
count = {}

print(f'Incremental class per step: {inc}')
for j in [0,1,2]:
    for e1, e2 in exp.items():
        file = f'{e2}/run_{j}_1993_.json'
        if not os.path.exists(file):
            continue
        result = json.load(open(file))

        top1_total = list(map(lambda x: 100 * x['accuracy']['total'], result['results']))

        if e1 in avg_top1_total:
            avg_top1_total[e1] += np.array(top1_total)
            cul_top1_total[e1].append(np.mean(top1_total))
            count[e1] += 1
        else:
            avg_top1_total[e1] = np.array(top1_total)
            cul_top1_total[e1] = [np.mean(top1_total)]
            count[e1] = 1
        
xs = list(range(3,6,inc))
for e1, e2 in avg_top1_total.items():
    top1_total = e2/count[e1]
    mean = statistics.mean(cul_top1_total[e1]) if len(cul_top1_total[e1])>1 else cul_top1_total[e1][0]
    std = statistics.stdev(cul_top1_total[e1]) if len(cul_top1_total[e1])>1 else 0
    if e1 != 'Ours':
        print(f"\t {e1}: {stats.ttest_rel(cul_top1_total['Ours'], cul_top1_total[e1])}")
    plt.plot(xs, top1_total, marker='o', label=f'{e1}: {mean:.1f}' + r' $\pm$ ' + f'{std:.1f}')

plt.xticks(xs)    
plt.xlabel('Number of Classes')
plt.ylabel('Accuracy')
#     plt.ylim(71,89)
plt.title(f'nfg3_ncls{inc}')
plt.grid(True)
plt.legend()
plt.show()

Forgetting (Table 2)

In [ ]:
folder = f'results/pretrain'
exps = {
    1:
       {
        'iCaRL': f'{folder}/icarl/20220403_icarl_diabetic_3steps1',
        'UCIR': f'{folder}/ucir/20220403_ucir_diabetic_3steps1',
        'PODNet': f'{folder}/podnet/20220403_podnet_cnn_diabetic_3steps1',
        'DER': f'{folder}/der/20220404_derE0_diabetic_3steps1',
        'Ours': f'{folder}/ours/20220403_derE3_diabetic_3steps1',
       },
      }

incs=[1]
row = min(len(incs),3)
plt.figure(figsize=(6*len(incs),6))

inc = 1
exp = exps[inc]
step = int(3//inc)

all_forget = {}
avg_forget = {}
count = {}

print(f'Incremental class per step: {inc}')
for j in [0,1,2]:
    for e1, e2 in exp.items():
        file = f'{e2}/run_{j}_1993_.json'
        if not os.path.exists(file):
            continue
        result = json.load(open(file))

        all_forget_total = {}
        for k in range(step,step*2):
            if k == step:
                start = 0
            else:
                start = (k-1)*inc
            end = k*inc - 1

            top1_first = np.zeros(len(result['results']) - (k-step))
            for p in range(start, end+1):
                new_key = f'{str(p).zfill(2)}-{str(p).zfill(2)}'
                top1_first += np.array(list(map(lambda x: x['accuracy_per_class'][new_key], result['results'][(k-step):])))
            top1_first /= (end-start+1) / 100

            forgetting = np.concatenate([[0], np.maximum.accumulate(top1_first)[:-1] - top1_first[1:]])

            if e1 in all_forget_total:
                all_forget_total[e1][k-step:] += np.array(forgetting)
            else:
                all_forget_total[e1] = np.array(forgetting)

        forgetting = all_forget_total[e1]        
        forgetting[1:] /= np.arange(len(forgetting)-1) + 1
        if e1 in all_forget:
            all_forget[e1] += forgetting
            avg_forget[e1].append(np.mean(forgetting))
            count[e1] += 1
        else:
            all_forget[e1] = forgetting
            avg_forget[e1] = [np.mean(forgetting)]
            count[e1] = 1

for e1, e2 in all_forget.items():
    forgetting = all_forget[e1]/count[e1]
    mean = statistics.mean(avg_forget[e1]) if len(avg_forget[e1])>1 else avg_forget[e1][0]
    std = statistics.stdev(avg_forget[e1]) if len(avg_forget[e1])>1 else 0
    if e1 != 'Ours':
        print(f"\t {e1}: {stats.ttest_rel(avg_forget['Ours'], avg_forget[e1])}")
    xs = list(range(3,6,inc))
    plt.plot(xs, forgetting, marker='o', label=f'{e1}: {mean:.1f}' + r' $\pm$ ' + f'{std:.1f}')
        
plt.xticks(xs)    
plt.xlabel('Number of Classes')
plt.ylabel('Forgetting')
#     plt.ylim(71,89)
plt.title(f'nfg3_ncls{inc}')
plt.grid(True)
plt.legend()

plt.show()


### HAM10000

Incremental Accuracy (Table 3)

In [ ]:
folder = f'results/pretrain'
exps = {
    1:
       {
        'iCaRL': f'{folder}/icarl/20220407_icarl_ham10000_3steps1',
        'UCIR': f'{folder}/ucir/20220407_ucir_ham10000_3steps1',
        'PODNet': f'{folder}/podnet/20220415_podnet_ham10000_3steps1',
        'DER': f'{folder}/der/20220415_derE0_ham10000_3steps1',
        'Ours': f'{folder}/ours/20220406_derE3_ham10000_3steps1',
       },
    2:
       {
        'iCaRL': f'{folder}/icarl/20220404_icarl_ham10000_3steps2',
        'UCIR': f'{folder}/ucir/20220404_ucir_ham10000_3steps2',
        'PODNet': f'{folder}/podnet/20220406_podnet_ham10000_3steps2',
        'DER': f'{folder}/der/20220405_derE0_ham10000_3steps2',
        'Ours': f'{folder}/ours/20220404_derE3_ham10000_3steps2',
       },
      }

incs=[1,2]
row = min(len(incs),3)
plt.figure(figsize=(6*len(incs),6))
for j, inc in enumerate(incs):
    print(f'Incremental class per step: {inc}')
    exp = exps[inc]
    plt.subplot(int(np.ceil(len(incs)//row)),row,j+1)

    avg_top1_total = {}
    cul_top1_total = {}
    count = {}
    for k in [0,1,2]:
        for e1, e2 in exp.items():
            file = f'{e2}/run_{k}_1993_.json'
            if not os.path.exists(file):
#                 file = f'{e2}/run_{k}_199_.json'
#                 if not os.path.exists(file):
                continue
            result = json.load(open(file))

            top1_total = list(map(lambda x: 100 * x['accuracy']['total'], result['results']))
        
            if e1 in avg_top1_total:
                avg_top1_total[e1] += np.array(top1_total)
                cul_top1_total[e1].append(np.mean(top1_total))
                count[e1] += 1
            else:
                avg_top1_total[e1] = np.array(top1_total)
                cul_top1_total[e1] = [np.mean(top1_total)]
                count[e1] = 1
        
    xs = list(range(3,8,inc))
    for e1, e2 in avg_top1_total.items():
        top1_total = e2/count[e1]
        mean = statistics.mean(cul_top1_total[e1]) if len(cul_top1_total[e1])>1 else cul_top1_total[e1][0]
        std = statistics.stdev(cul_top1_total[e1]) if len(cul_top1_total[e1])>1 else 0
        if e1 != 'Ours':
            print(f"\t {e1}: {stats.ttest_rel(cul_top1_total['Ours'], cul_top1_total[e1])}")
        plt.plot(xs, top1_total, marker='o', label=f'{e1}: {mean:.1f}' + r' $\pm$ ' + f'{std:.1f}')

    # plt.plot(xs, test)

    plt.xlabel('Number of Classes')
    plt.xticks(list(range(3,8)))
    plt.ylabel('Accuracy')
#     plt.ylim(71,89)
    plt.title(f'nfg3_ncls{inc}')
    plt.grid(True)
    plt.legend()
plt.show()

Forgetting (Table 3)

In [ ]:
folder = f'results/pretrain'
exps = {
    1:
       {
        'iCaRL': f'{folder}/icarl/20220407_icarl_ham10000_3steps1',
        'UCIR': f'{folder}/ucir/20220407_ucir_ham10000_3steps1',
        'PODNet': f'{folder}/podnet/20220415_podnet_ham10000_3steps1',
        'DER': f'{folder}/der/20220415_derE0_ham10000_3steps1',
        'Ours': f'{folder}/ours/20220406_derE3_ham10000_3steps1',
       },
    2:
       {
        'iCaRL': f'{folder}/icarl/20220404_icarl_ham10000_3steps2',
        'UCIR': f'{folder}/ucir/20220404_ucir_ham10000_3steps2',
        'PODNet': f'{folder}/podnet/20220406_podnet_ham10000_3steps2',
        'DER': f'{folder}/der/20220405_derE0_ham10000_3steps2',
        'Ours': f'{folder}/ours/20220404_derE3_ham10000_3steps2',
       },
      }

incs=[1,2]
row = min(len(incs),3)
plt.figure(figsize=(6*len(incs),6))
for j, inc in enumerate(incs):
    print(f'Incremental class per step: {inc}')
    exp = exps[inc]
    step = int(3//inc)

    plt.subplot(int(np.ceil(len(incs)//row)),row,j+1)

    all_forget = {}
    avg_forget = {}
    count = {}
    for j in [0,1,2]:
        for e1, e2 in exp.items():
            file = f'{e2}/run_{j}_1993_.json'
            if not os.path.exists(file):
                continue
            result = json.load(open(file))
            
            all_forget_total = {}
            for k in range(step,step*2+2):
                if k == step:
                    start = 0
                else:
                    start = (k-1)*inc
                end = k*inc - 1

                top1_first = np.zeros(len(result['results']) - (k-step))
                for p in range(start, end+1):
                    new_key = f'{str(p).zfill(2)}-{str(p).zfill(2)}'
                    top1_first += np.array(list(map(lambda x: x['accuracy_per_class'][new_key], result['results'][(k-step):])))
                top1_first /= (end-start+1) / 100

                forgetting = np.concatenate([[0], np.maximum.accumulate(top1_first)[:-1] - top1_first[1:]])

                if e1 in all_forget_total:
                    all_forget_total[e1][k-step:] += np.array(forgetting)
                else:
                    all_forget_total[e1] = np.array(forgetting)
                    
            forgetting = all_forget_total[e1]        
            forgetting[1:] /= np.arange(len(forgetting)-1) + 1
            if e1 in all_forget:
                all_forget[e1] += forgetting
                avg_forget[e1].append(np.mean(forgetting))
                count[e1] += 1
            else:
                all_forget[e1] = forgetting
                avg_forget[e1] = [np.mean(forgetting)]
                count[e1] = 1
            
    for e1, e2 in all_forget.items():
        forgetting = all_forget[e1]/count[e1]
        mean = statistics.mean(avg_forget[e1]) if len(avg_forget[e1])>1 else avg_forget[e1][0]
        std = statistics.stdev(avg_forget[e1]) if len(avg_forget[e1])>1 else 0
        if e1 != 'Ours':
            print(f"\t {e1}: {stats.ttest_rel(avg_forget['Ours'], avg_forget[e1])}")
        xs = list(range(3,8,inc))
        plt.plot(xs, forgetting, marker='o', label=f'{e1}: {mean:.1f}' + r' $\pm$ ' + f'{std:.1f}')
        
    plt.xlabel('Number of Classes')
    plt.xticks(list(range(3,8)))
    plt.ylabel('Forgetting')
#     plt.ylim(71,89)
    plt.title(f'nfg3_ncls{inc}')
    plt.grid(True)
    plt.legend()

plt.show()
